<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/Snippets_Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from __future__ import absolute_import, print_function
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import glob, os
from os.path import *
from pathlib import Path

drive=None
fileIdDictionary={}
directoryListDict={}
allFilesList=[]
directoryList=[]
fileList=[]

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

def mountGoogleDrive():
    '''ToDo...
    fix help
    '''
    from google.colab import drive as gdrive
    gdrive.mount('/content/drive', force_remount=True)
    # contentPath = Path.cwd()
    allFilesList = glob.glob('**', recursive=True)
    print(f'{len(allFilesList)} files in Google Drive')
    os.chdir('/content/drive/My Drive')
    # myDrivePath = Path.cwd()
    # import BashColors
    from BashColors import C
    print(f'cwd: {C.Green}{Path.cwd()}{C.ColorOff}\n')
    

def listGoogleDriveFiles(silent=True):
    '''List all files in the root
    returns fileIdDictionary
    Search query reference:
    https://developers.google.com/drive/v2/web/search-parameters'''
    
    fileList = drive.ListFile({'q': "title contains '.*' and 'root' in parents"}).GetList() 
    counter=0
    for fil in fileList:
        fileName = fil['title']
        file_id = fil['id']
        fileId = fil['id']
        fileMimeType = fil['mimeType']
        fullPath = abspath(fileName)
        counter+=1
        fileIdDictionary[counter]=(fileName, fil['id'], fullPath)
        if fileMimeType=='application/vnd.google-apps.folder':
            directoryList.append(fullPath)
            directoryListDict[fileName]=(fil['id'], fullPath)


        if not silent:
            print(f'{counter}. {fileName}')
            print(f'{fileMimeType}')
            print(f'{file_id}\n')
            # if fileMimeType=='application/vnd.google-apps.folder':


mountGoogleDrive()
listGoogleDriveFiles(silent=False)

In [ ]:
def selectFile(itemNumber:int):
    '''Returns selected file id'''
    counter=0
    for item in fileIdDictionary.keys():
        counter+=1
        dict_item = fileIdDictionary[item]
        print(counter, dict_item[0])

    selection = fileIdDictionary[itemNumber]
    selectName = selection[0]
    selectId = selection[1]
    print(f'{selectName}')
    return selectId


idd=selectFile(itemNumber=1)
print(idd)

# Saving data to Google Drive

In [ ]:

def createAndUploadFile(named:str, inDir:str, contentStr:str):
    '''Create & upload a text file.'''
    currentPath=Path.cwd()
    dir_path=join('/content/drive/My Drive', inDir)
    if exists(dir_path):
        os.chdir(dir_path)
        print(Path.cwd)
    else:
        os.mkdir(dir_path)
        os.chdir(dir_path)
    uploaded = drive.CreateFile({'title': 'Sample file.txt'})
    uploaded.SetContentString('Sample upload file content')
    uploaded.Upload()
    listGoogleDriveFiles(silent=False)
    print('Uploaded file with ID {}'.format(uploaded.get('id')))


In [ ]:
fileContent='New content here.'
createAndUploadFile(named='bullshit.py', inDir='testDir', contentStr=fileContent)

In [ ]:
def downloadGoogleDriveFile(thisFileId):
    '''Download a file based on its file id'''
    print(thisFileId)
    file_id = thisFileId
    downloaded = drive.CreateFile({'id': file_id})
    print('Downloaded content "{}"'.format(downloaded.GetContentString()))

In [ ]:
downloadFile = selectFile(13)
downloadGoogleDriveFile(downloadFile)